<div style="border: 2px solid #2196F3; padding: 15px; border-radius: 10px; background-color: #e3f2fd; text-align: center; color: #000; font-weight: bold;">

# **Retail Inventory Management System - Llama-index Implementation** 

</div>



In [6]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not set.")

# Load PDF documents
documents = SimpleDirectoryReader(input_files=["retail.pdf"]).load_data()

# Create a vector index for querying
index = VectorStoreIndex.from_documents(documents)

# Create a query engine
query_engine = index.as_query_engine()

# Extract requirements from the PDF
response = query_engine.query("Summarize the key requirements for the Retail Inventory Management System.")
customer_requirements = response.response
print("Extracted Requirements:", customer_requirements)

Extracted Requirements: The key requirements for the Retail Inventory Management System are detailed on pages 13 and 14 of the retail.pdf file.


## **Initialize LLM for LangChain**

We use OpenAI’s `gpt-4o-mini` model to power our agents in LangChain:

```python
# Initialize LLM for LangChain
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)


In [7]:
# Initialize LLM for LangChain
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

## **LangChain Agent Initialization and Role Prompts**

We start by importing the necessary modules and initializing the LLM used across all agents.


In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=api_key)

# Common estimation instructions to be appended to every agent prompt
ESTIMATION_INSTRUCTIONS = """
After completing your primary response, please provide a detailed estimate of the work/time required for your deliverables. Use the following format for your estimation:

### Total Estimated Requirements 
Based on the above user stories, we can estimate approximately **10–12 requirements** including non-functional ones, like performance and security requirements.

### Effort Estimates
- **Total requirements:** [your estimated number]
- **Daily productivity rate:** [your estimated rate] requirements/day
- **Development time estimate:** 
   - [your estimated number] requirements ÷ [your estimated rate] requirements/day ≈ [calculated days] days (rounded up to [rounded development days] days)
- **Review and rework cycles:** 
   - Estimated at [estimated review days] day(s) for peer reviews and fixing potential issues.

### Summary of Effort
- **Total Development Effort:** [rounded development days] days
- **Review and Rework Effort:** [estimated review days] day(s)
- **Total Estimated Timeline:** **[sum of above] days**
"""

# Updated Agent Prompts with estimation instructions appended

CUSTOMER_PROMPT = f"""You are a Customer Agent for a retail business. Your role is to provide clear requirements for the Retail Inventory Management System based on provided details. Relay these requirements to the Product Owner and review sprint deliverables, providing feedback for improvements. {ESTIMATION_INSTRUCTIONS}
Requirements: {{requirements}}"""

PRODUCT_OWNER_PROMPT = f"""You are a Product Owner in a Scrum team. Translate customer requirements into a prioritized product backlog consisting of user stories. Collaborate with the Scrum Master to define sprint goals and incorporate customer feedback after each sprint. Deliverables: product backlog, sprint goals. {ESTIMATION_INSTRUCTIONS}"""

SCRUM_MASTER_PROMPT = f"""You are a Scrum Master facilitating Agile ceremonies. Create sprint backlogs based on the product backlog, ensure team collaboration, and remove blockers. Deliverables: sprint backlog, sprint summary. {ESTIMATION_INSTRUCTIONS}"""

DEVELOPER_PROMPT = f"""You are a Developer Agent. Implement features from the sprint backlog by writing modular code for the Retail Inventory Management System. Deliverables: feature implementations, progress report. {ESTIMATION_INSTRUCTIONS}"""

QA_ENGINEER_PROMPT = f"""You are a QA Engineer Agent. Develop and execute test cases for the sprint deliverables to ensure functionality and quality. Deliverables: test cases, test results, defect report. {ESTIMATION_INSTRUCTIONS}"""

UI_UX_DESIGNER_PROMPT = f"""You are a UI/UX Designer Agent. Create wireframes and design prototypes for the sprint's user stories, ensuring usability and accessibility. Deliverables: wireframes, design prototypes. {ESTIMATION_INSTRUCTIONS}"""

BUSINESS_ANALYST_PROMPT = f"""You are a Business Analyst Agent. Refine user stories by adding detailed requirements and acceptance criteria. Deliverables: refined user stories, requirements document. {ESTIMATION_INSTRUCTIONS}"""

DEVOPS_ENGINEER_PROMPT = f"""You are a DevOps Engineer Agent. Set up CI/CD pipelines and deploy sprint deliverables efficiently. Deliverables: deployment plan, pipeline configuration. {ESTIMATION_INSTRUCTIONS}"""

ARCHITECT_PROMPT = f"""You are an Architect Agent. Design the system architecture for sprint features, ensuring the solution is scalable and reliable. Deliverables: architecture diagram, technical specifications. {ESTIMATION_INSTRUCTIONS}"""

SECURITY_ENGINEER_PROMPT = f"""You are a Security Engineer Agent. Audit the sprint deliverables for vulnerabilities and provide mitigations. Deliverables: security report, mitigation plan. {ESTIMATION_INSTRUCTIONS}"""

TECHNICAL_WRITER_PROMPT = f"""You are a Technical Writer Agent. Document the sprint deliverables, including user guides and release notes. Provide a final consolidated report after three sprints. Deliverables: documentation, release summary. {ESTIMATION_INSTRUCTIONS}"""


## **ScrumState Data Structure**

We define a `ScrumState` class using Python’s `TypedDict` to track the state across all phases of the Scrum simulation:

In [9]:
from typing import TypedDict, List, Dict, Optional

class ScrumState(TypedDict):
    customer_requirements: str
    product_backlog: str
    sprint_backlog: str
    sprint_number: int
    sprint_deliverables: Dict[str, str]
    customer_feedback: str
    current_phase: str
    conversation_history: List[Dict[str, str]]
    completion_status: str

## 🚦 Scrum Phase Functions

These functions define each stage of the simulated Scrum cycle using LLM agents.


In [10]:
def customer_phase(state: ScrumState) -> ScrumState:
    messages = [
        SystemMessage(content=CUSTOMER_PROMPT.format(requirements=state["customer_requirements"])),
        HumanMessage(content="Provide the requirements for the Retail Inventory Management System.")
    ]
    response = llm.invoke(messages)
    state["customer_requirements"] = response.content
    state["conversation_history"].append({"Customer": response.content})
    state["current_phase"] = "product_owner"
    print("Customer Requirements:", response.content)
    return state

def product_owner_phase(state: ScrumState) -> ScrumState:
    messages = [
        SystemMessage(content=PRODUCT_OWNER_PROMPT),
        HumanMessage(content=f"Create a product backlog based on these requirements:\n{state['customer_requirements']}")
    ]
    response = llm.invoke(messages)
    state["product_backlog"] = response.content
    state["conversation_history"].append({"Product Owner": response.content})
    state["current_phase"] = "scrum_master"
    print("Product Backlog:", response.content)
    return state

def scrum_master_phase(state: ScrumState) -> ScrumState:
    messages = [
        SystemMessage(content=SCRUM_MASTER_PROMPT),
        HumanMessage(content=f"Create a sprint backlog for sprint {state['sprint_number']} based on this product backlog:\n{state['product_backlog']}")
    ]
    response = llm.invoke(messages)
    state["sprint_backlog"] = response.content
    state["conversation_history"].append({"Scrum Master": response.content})
    state["current_phase"] = "development"
    print(f"Sprint {state['sprint_number']} Backlog:", response.content)
    return state

def development_phase(state: ScrumState) -> ScrumState:
    team_agents = [
        ("Developer", DEVELOPER_PROMPT),
        ("QA Engineer", QA_ENGINEER_PROMPT),
        ("UI/UX Designer", UI_UX_DESIGNER_PROMPT),
        ("Business Analyst", BUSINESS_ANALYST_PROMPT),
        ("DevOps Engineer", DEVOPS_ENGINEER_PROMPT),
        ("Architect", ARCHITECT_PROMPT),
        ("Security Engineer", SECURITY_ENGINEER_PROMPT),
        ("Technical Writer", TECHNICAL_WRITER_PROMPT)
    ]
    state["sprint_deliverables"] = {}
    for agent_name, prompt in team_agents:
        messages = [
            SystemMessage(content=prompt),
            HumanMessage(content=f"Perform your role for sprint {state['sprint_number']} based on this sprint backlog:\n{state['sprint_backlog']}")
        ]
        response = llm.invoke(messages)
        state["sprint_deliverables"][agent_name] = response.content
        state["conversation_history"].append({agent_name: response.content})
        print(f"{agent_name} Deliverables:", response.content)
    state["current_phase"] = "customer_review"
    return state

def customer_review_phase(state: ScrumState) -> ScrumState:
    deliverables_summary = "\n".join([f"{agent}: {content}" for agent, content in state["sprint_deliverables"].items()])
    messages = [
        SystemMessage(content=CUSTOMER_PROMPT.format(requirements=state["customer_requirements"])),
        HumanMessage(content=f"Review the deliverables for sprint {state['sprint_number']}:\n{deliverables_summary}\nProvide feedback.")
    ]
    response = llm.invoke(messages)
    state["customer_feedback"] = response.content
    state["conversation_history"].append({"Customer Feedback": response.content})
    state["sprint_number"] += 1
    # After 3 sprints, go to final_report, not "completed"
    state["current_phase"] = "product_owner" if state["sprint_number"] <= 3 else "final_report"
    print(f"Customer Feedback (Sprint {state['sprint_number']-1}):", response.content)
    return state

def final_report_phase(state: ScrumState) -> ScrumState:
    history_summary = "\n".join([f"{key}: {value}" for entry in state["conversation_history"] for key, value in entry.items()])
    messages = [
        SystemMessage(content=TECHNICAL_WRITER_PROMPT.format(sprint_number="final")),
        HumanMessage(content=f"Generate a final report summarizing the three sprints based on this history:\n{history_summary}")
    ]
    response = llm.invoke(messages)
    print("\n=== Final Report for Three Sprints ===")
    print(response.content)
    state["conversation_history"].append({"Technical Writer Final Report": response.content})
    state["completion_status"] = "completed"  # Explicitly set to exit the loop
    return state

## 🧠 Run the Scrum Workflow

This function initializes the `ScrumState` and iteratively transitions through the Scrum phases until completion. Each phase represents a part of the agile lifecycle and is executed in sequence over 3 sprints.

In [11]:
def run_workflow():
    # Initialize state
    initial_state = ScrumState(
        customer_requirements=customer_requirements,  # From LlamaIndex
        product_backlog="",
        sprint_backlog="",
        sprint_number=1,
        sprint_deliverables={},
        customer_feedback="",
        current_phase="customer",
        conversation_history=[],
        completion_status="in_progress"
    )

    state = initial_state
    while state["completion_status"] != "completed":
        if state["current_phase"] == "customer":
            state = customer_phase(state)
        elif state["current_phase"] == "product_owner":
            state = product_owner_phase(state)
        elif state["current_phase"] == "scrum_master":
            state = scrum_master_phase(state)
        elif state["current_phase"] == "development":
            state = development_phase(state)
        elif state["current_phase"] == "customer_review":
            state = customer_review_phase(state)
        elif state["current_phase"] == "final_report":
            state = final_report_phase(state)
            # No need to set completion_status here again; it’s done in the phase
        else:
            print(f"Unexpected phase: {state['current_phase']}, terminating workflow")
            state["completion_status"] = "completed"  # Failsafe for unexpected states
            break

    print("Workflow completed after 3 sprints!")
    return state

# Run the workflow
final_state = run_workflow()

Customer Requirements: Based on the provided details and the context of a Retail Inventory Management System, here are the key requirements:

### Functional Requirements
1. **Inventory Tracking**: The system must allow users to track inventory levels in real-time, including stock counts, product details, and location within the store or warehouse.
2. **Product Management**: Users should be able to add, update, and delete product information, including SKU, description, price, and category.
3. **Supplier Management**: The system must enable users to manage supplier information, including contact details, product offerings, and order history.
4. **Order Management**: Users should be able to create, manage, and track purchase orders and sales orders, including order status updates.
5. **Reporting and Analytics**: The system should provide reporting features to analyze inventory turnover, sales trends, and stock levels.
6. **Barcode Scanning**: The system must support barcode scanning for 